In [1]:
import datetime

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.metrics import MeanAbsolutePercentageError, MeanAbsoluteError, RootMeanSquaredError

import definitions
from training import train
from training.loguniform import LogUniform
from training.stepuniform import StepUniform
from training.steploguniform import StepLogUniform
from scipy.stats.distributions import randint
import numpy as np

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [2]:
def build_model(hparams):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(13,)))
    for _ in range(hparams['num_layers']):
        model.add(layers.Dense(units=hparams['num_units'],
                               activation='relu'))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hparams['learning_rate']),
        loss='mean_squared_error',
        metrics=[MeanAbsolutePercentageError(), MeanAbsoluteError(), RootMeanSquaredError()])
    return model

In [3]:
log_dir = definitions.LOG_DIR / 'Wlnu' / 'test'
hp_rv = {'num_layers': randint(2, 5),
            'num_units': StepUniform(start=128, num=10, step=128),
            'learning_rate': LogUniform(loc=-4, scale=2, base=10, discrete=False),
            'batch_size': StepLogUniform(start=5, num=4, step=1, base=2),
            'epochs': randint(100, 1000)}
print(log_dir)

C:\Users\alexj\work\mass_regression\logs\Wlnu\test


In [26]:
(x, y), (x_val, y_val) = keras.datasets.boston_housing.load_data()
print(y)

[15.2 42.3 50.  21.1 17.7 18.5 11.3 15.6 15.6 14.4 12.1 17.9 23.1 19.9
 15.7  8.8 50.  22.5 24.1 27.5 10.9 30.8 32.9 24.  18.5 13.3 22.9 34.7
 16.6 17.5 22.3 16.1 14.9 23.1 34.9 25.  13.9 13.1 20.4 20.  15.2 24.7
 22.2 16.7 12.7 15.6 18.4 21.  30.1 15.1 18.7  9.6 31.5 24.8 19.1 22.
 14.5 11.  32.  29.4 20.3 24.4 14.6 19.5 14.1 14.3 15.6 10.5  6.3 19.3
 19.3 13.4 36.4 17.8 13.5 16.5  8.3 14.3 16.  13.4 28.6 43.5 20.2 22.
 23.  20.7 12.5 48.5 14.6 13.4 23.7 50.  21.7 39.8 38.7 22.2 34.9 22.5
 31.1 28.7 46.  41.7 21.  26.6 15.  24.4 13.3 21.2 11.7 21.7 19.4 50.
 22.8 19.7 24.7 36.2 14.2 18.9 18.3 20.6 24.6 18.2  8.7 44.  10.4 13.2
 21.2 37.  30.7 22.9 20.  19.3 31.7 32.  23.1 18.8 10.9 50.  19.6  5.
 14.4 19.8 13.8 19.6 23.9 24.5 25.  19.9 17.2 24.6 13.5 26.6 21.4 11.9
 22.6 19.6  8.5 23.7 23.1 22.4 20.5 23.6 18.4 35.2 23.1 27.9 20.6 23.7
 28.  13.6 27.1 23.6 20.6 18.2 21.7 17.1  8.4 25.3 13.8 22.2 18.4 20.7
 31.6 30.5 20.3  8.8 19.2 19.4 23.1 23.  14.8 48.8 22.6 33.4 21.1 13.6
 32.2 13.1

In [5]:
mean = np.mean(x)
std = np.std(x)
print(mean)
print(std)

69.79277358530084
144.39194712410696


In [6]:
x = (x - mean) / std
x_val = (x_val - mean) / std
x.shape

(404, 13)

In [7]:
train.random_search(build_fn=build_model, x=x, y=y, x_val=x_val, y_val=y_val, n=10, hp_rv=hp_rv, log_dir=log_dir)

absolute_error: 2.3473 - root_mean_squared_error: 3.2778 - val_loss: 22.3511 - val_mean_absolute_percentage_error: 15.4739 - val_mean_absolute_error: 3.1510 - val_root_mean_squared_error: 4.7295
Epoch 626/684
404/404 [==============================] - 0s 114us/sample - loss: 9.2440 - mean_absolute_percentage_error: 10.7049 - mean_absolute_error: 2.1986 - root_mean_squared_error: 3.0398 - val_loss: 23.4899 - val_mean_absolute_percentage_error: 14.5402 - val_mean_absolute_error: 3.1116 - val_root_mean_squared_error: 4.8475
Epoch 627/684
404/404 [==============================] - 0s 114us/sample - loss: 10.6024 - mean_absolute_percentage_error: 11.3822 - mean_absolute_error: 2.4240 - root_mean_squared_error: 3.2557 - val_loss: 23.5821 - val_mean_absolute_percentage_error: 15.3467 - val_mean_absolute_error: 3.2869 - val_root_mean_squared_error: 4.8558
Epoch 628/684
404/404 [==============================] - 0s 109us/sample - loss: 11.0186 - mean_absolute_percentage_error: 10.5427 - mean_ab

In [9]:
model = tf.keras.models.load_model(str(log_dir / 'best_model.h5'))

In [23]:
y_pred = model.predict(x_val)
print(y_pred[:, 0].shape)

(102,)


In [16]:
import altair as alt
from vega_datasets import data

import pandas as pd

In [35]:
chart_data = pd.DataFrame({'Price': np.concatenate((y_pred[:, 0], y_val)), 'Type': ['Estimate']*y_pred.shape[0] + ['Actual']*y_pred.shape[0]})
print(chart_data)

Price      Type
0    10.351562  Estimate
1    19.125000  Estimate
2    20.953125  Estimate
3    36.812500  Estimate
4    24.265625  Estimate
..         ...       ...
199  21.900000    Actual
200  24.100000    Actual
201  50.000000    Actual
202  26.700000    Actual
203  25.000000    Actual

[204 rows x 2 columns]


In [40]:
alt.Chart(chart_data).mark_bar().encode(alt.X("Price:Q", bin=True), y="count()", color="Type")

alt.Chart(...)

In [41]:
difference = y_pred[:, 0] - y_val

In [42]:
chart_data = pd.DataFrame({'Actual - Expected': difference})
print(chart_data)

Actual - Expected
0             3.151562
1             0.325000
2             1.953125
3             9.812500
4             2.065625
..                 ...
97           19.568750
98           -0.990625
99           -3.718750
100          -1.059375
101          -1.828125

[102 rows x 1 columns]


In [45]:
alt.Chart(chart_data).mark_bar().encode(alt.X("Actual - Expected:Q", bin=alt.Bin(step=2)), y="count()")

alt.Chart(...)